In [ ]:
!pip install rdflib

In [ ]:
from rdflib import Graph, Namespace, RDF, RDFS, XSD
import networkx as nx
import matplotlib.pyplot as plt

## Start to define the TBox

In [ ]:
g = Graph()
pub = Namespace("http://www.example.org/publication#")
g.bind("pub", pub)

Here we will define the class and the subclass of the schema

In [ ]:
# Class
g.add((pub.Scientist, RDF.type, RDFS.Class))
g.add((pub.Publication, RDF.type, RDFS.Class))
g.add((pub.Review, RDF.type, RDFS.Class))
g.add((pub.Topic, RDF.type, RDFS.Class))
g.add((pub.Event, RDF.type, RDFS.Class))
g.add((pub.Authors, RDF.type, RDFS.Class))
#Subclass
g.add((pub.RevelantAuthors, RDFS.subClassOf, pub.Authors))
g.add((pub.Reviewer, RDFS.subClassOf, pub.RevelantAuthors))
g.add((pub.JournalEditor, RDFS.subClassOf, pub.Scientist))
g.add((pub.Conference, RDFS.subClassOf, pub.Event))
g.add((pub.Workshop, RDFS.subClassOf, pub.Event))
g.add((pub.Conferencechair, RDFS.subClassOf, pub.Scientist))


In [ ]:
g.add((pub.Title, RDF.type, RDF.Property))
g.add((pub.Title, RDFS.domain, pub.Paper))
g.add((pub.Title, RDFS.range, XSD.string))

g.add((pub.Abstract, RDF.type, RDF.Property))
g.add((pub.Abstract, RDFS.domain, pub.Paper))
g.add((pub.Abstract, RDFS.range, XSD.string))

g.add((pub.Has_topic, RDF.type, RDF.Property))
g.add((pub.Has_topic, RDFS.domain, pub.Paper))
g.add((pub.Has_topic, RDFS.range, pub.Topic))

g.add((pub.review, RDF.type, RDF.Property))
g.add((pub.review, RDFS.domain, pub.Publication))
g.add((pub.review, RDFS.range, pub.Review))

g.add((pub.reviewer, RDF.type, RDF.Property))
g.add((pub.reviewer, RDFS.domain, pub.Review))
g.add((pub.reviewer, RDFS.range, pub.Reviewer))

g.add((pub.reviewedPaper, RDF.type, RDF.Property))
g.add((pub.reviewedPaper, RDFS.domain, pub.Review))
g.add((pub.reviewedPaper, RDFS.range, pub.Paper))

g.add((pub.presented_in, RDF.type, RDF.Property))
g.add((pub.presented_in, RDFS.domain, pub.Publication))
g.add((pub.presented_in, RDFS.range, pub.Proceeding))

g.add((pub.published_in, RDF.type, RDF.Property))
g.add((pub.published_in, RDFS.domain, pub.Publication))
g.add((pub.published_in, RDFS.range, pub.Volume))

g.add((pub.belongs_to, RDF.type, RDF.Property))
g.add((pub.belongs_to, RDFS.domain, pub.Proceeding))
g.add((pub.belongs_to, RDFS.range, pub.Event))
g.add((pub.belongs_to, RDFS.domain, pub.Volume))
g.add((pub.belongs_to, RDFS.range, pub.Journal))

g.add((pub.has_year, RDF.type, RDF.Property))
g.add((pub.year, RDFS.domain, pub.Proceeding))
g.add((pub.year, RDFS.range, pub.int))
g.add((pub.year, RDFS.domain, pub.Volume))
g.add((pub.year, RDFS.range, pub.int))


g.add((pub.city, RDFS.domain, pub.Proceeding))
g.add((pub.city, RDFS.range, pub.string))
g.add((pub.city, RDFS.domain, pub.Volume))
g.add((pub.city, RDFS.range, pub.string))

g.add((pub.cities, RDFS.domain, pub.Publication))
g.add((pub.cities, RDFS.range, pub.Publication))


## Save in the file rdfs

In [ ]:
g.serialize("tbox_lab.ttl", format="turtle")

In [ ]:
from rdflib import Graph, RDF, RDFS
import matplotlib.pyplot as plt
import networkx as nx

g = Graph()
g.parse("tbox_lab.ttl", format="turtle")

G = nx.DiGraph()

for s, _, _ in g.triples((None, RDF.type, RDFS.Class)):
    label = str(s).split("#")[-1]
    G.add_node(label, layer="Class", color="mediumpurple")

for s, _, _ in g.triples((None, RDF.type, RDF.Property)):
    domain = g.value(s, RDFS.domain)
    range_ = g.value(s, RDFS.range)
    if domain and range_:
        prop = str(s).split("#")[-1]
        domain_label = str(domain).split("#")[-1]
        range_label = str(range_).split("#")[-1]
        G.add_node(prop, layer="Property", color="lightsteelblue")
        G.add_edge(prop, domain_label, label="rdfs:domain")
        G.add_edge(prop, range_label, label="rdfs:range")

for subclass, _, superclass in g.triples((None, RDFS.subClassOf, None)):
    sub_label = str(subclass).split("#")[-1]
    super_label = str(superclass).split("#")[-1]
    G.add_node(sub_label, layer="Class", color="mediumpurple")
    G.add_node(super_label, layer="Class", color="mediumpurple")
    G.add_edge(sub_label, super_label, label="rdfs:subClassOf")

pos = nx.spring_layout(G, seed=42, k=2.5)
edge_labels = nx.get_edge_attributes(G, 'label')
node_colors = [G.nodes[n].get('color', 'lightgray') for n in G.nodes]

plt.figure(figsize=(22, 16))
nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=3000)
nx.draw_networkx_labels(G, pos, font_size=10, font_weight='bold')
nx.draw_networkx_edges(G, pos, arrows=True)
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='blue', font_size=9)

plt.title("RDF schema (TBOX) ", fontsize=18)
plt.axis('off')
plt.tight_layout()
plt.show()

